In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 20:07:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
input_file = "yellow_tripdata_2024-10.parquet"
df = spark.read.parquet(input_file)
df_repartitioned = df.repartition(4) # repartition
df_repartitioned.write.parquet(r"./pq/")

In [9]:
# calculate avarage size of the parquet
import os
import glob

pq_files = glob.glob(os.path.join(r"./pq", "*.parquet"))

file_sizes = [os.path.getsize(f) for f in pq_files]

avg_size = sum(file_sizes) / len(file_sizes)
avg_size = avg_size / (1024 * 1024) # convert to MB
print(round(avg_size, 2))

22.4


In [12]:
cnt = df_repartitioned \
    .filter(
        (df_repartitioned.tpep_pickup_datetime >= '2024-10-15 00:00:00') &
        (df_repartitioned.tpep_pickup_datetime < '2024-10-16 00:00:00')
        ) \
    .count()

print(cnt)

128893


In [14]:
df_repartitioned.registerTempTable("trips_data")

/Users/haozhewang/Desktop/DataEngineeringZoomCamp/.conda/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [16]:
df_longest_trip = spark.sql(
    """
    SELECT 
        (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS hour_difference
    FROM
        trips_data
    ORDER BY 1 DESC
    LIMIT 5
    """
)

df_longest_trip.show()

+------------------+
|   hour_difference|
+------------------+
|162.61777777777777|
|           143.325|
|137.76055555555556|
|114.83472222222223|
| 89.89833333333333|
+------------------+



In [19]:
df_zone = spark.read \
    .option("header", "true") \
    .csv("taxi_zone_lookup.csv")

In [23]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
df_zone.registerTempTable("zones")

/Users/haozhewang/Desktop/DataEngineeringZoomCamp/.conda/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [34]:
df_result = spark.sql(
    """
    SELECT 
        trips_data.PULocationID AS pickup_locationID,
        zones.Zone AS zone_name, 
        COUNT(1) as frequency
    FROM
        trips_data
    INNER JOIN zones on trips_data.PULocationID = zones.LocationID
    GROUP BY 1, 2
    ORDER BY 3 ASC
    LIMIT 5
    """
)

df_result.show(truncate=False)


+-----------------+---------------------------------------------+---------+
|pickup_locationID|zone_name                                    |frequency|
+-----------------+---------------------------------------------+---------+
|105              |Governor's Island/Ellis Island/Liberty Island|1        |
|5                |Arden Heights                                |2        |
|199              |Rikers Island                                |2        |
|2                |Jamaica Bay                                  |3        |
|111              |Green-Wood Cemetery                          |3        |
+-----------------+---------------------------------------------+---------+

